In [ ]:
import netCDF4
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from datetime import timedelta as delta

fname = '00000003.nc'

In [ ]:
data_netcdf4 = netCDF4.Dataset(fname)
print(data_netcdf4)

In [ ]:
trajectory_netcdf4 = data_netcdf4.variables['trajectory'][:]
time_netcdf4 = data_netcdf4.variables['time'][:]
lon_netcdf4 = data_netcdf4.variables['lon'][:]
lat_netcdf4 = data_netcdf4.variables['lat'][:]

In [ ]:
data_xarray = xr.open_dataset(fname)
print(data_xarray)

In [ ]:
np.set_printoptions(linewidth=160)
ns_per_hour = np.timedelta64(1, 'h') # nanoseconds in an hour

In [ ]:
x = data_xarray['lon'].values
y = data_xarray['lat'].values
distance = np.cumsum(np.sqrt(np.square(np.diff(x))+np.square(np.diff(y))),axis=1)  # d = (dx^2 + dy^2)^(1/2)

real_time = data_xarray['time']
#real_time = time_netcdf4 # convert time to hours
time_since_release = (real_time.values.transpose() - real_time.values[:,0])/np.timedelta64(1, 'D') # substract the initial time from each timeseries


In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,4),constrained_layout=True)

ax1.set_ylabel('Distance travelled [m]')
ax1.set_xlabel('observation',weight='bold')
d_plot = ax1.plot(distance.transpose())

ax2.set_ylabel('Distance travelled [m]')
ax2.set_xlabel('time since release [hours]',weight='bold')
d_plot_t = ax2.plot(time_since_release[1:],distance.transpose())
plt.show()

In [ ]:
import cartopy.crs as ccrs

extent = [28, 29, 43, 44.5]

def start_axes(title, extent=extent, fig=None, sp=None):
    if fig is None:
        fig = plt.figure(figsize=(13, 5))
        
    if sp is None:
        ax = fig.add_axes([0.03, 0.03, 0.90, 0.94],projection=ccrs.PlateCarree())
    else:
        ax = fig.add_subplot(sp,projection=ccrs.PlateCarree())
            
    ax.set_extent(extent)
    ax.gridlines()
    ax.coastlines(resolution='10m')
    #ax.set_xlim(-6, 36.5), ax.set_ylim(30, 46)
    #ax.set_aspect("equal")
    ax.set_title(title)
    ax.gridlines(xlocs=range(25,42,1), ylocs=range(40,48,1),draw_labels=True)#
    return ax


In [ ]:
outputdt = delta(hours=6)
timerange = np.arange(np.nanmin(data_xarray['time'].values),
                      np.nanmax(data_xarray['time'].values)+np.timedelta64(outputdt), 
                      outputdt) # timerange in nanoseconds

In [ ]:
t = timerange[0]
np.datetime_as_string(t, unit='m')

In [ ]:
from matplotlib.animation import FuncAnimation

# %%capture
fig = plt.figure(figsize=(10,5))

ax1=start_axes('test', fig=fig,sp='121', extent=[28, 29.5, 43, 44.5])
ax2=start_axes('test', fig=fig,sp='122', extent=[28.5, 29, 43.8, 44.5])

time_id = np.where(data_xarray['time'] == timerange[0]) # Indices of the data where time = 0
scat1 = ax1.scatter(data_xarray['lon'].values[time_id],
                     data_xarray['lat'].values[time_id],10,
                    time_since_release.transpose()[time_id], vmin=0,vmax=10)

#clb1 =plt.colorbar(scat1)
#clb1.ax.set_xlabel('Age [d]')

scat2 = ax2.scatter(data_xarray['lon'].values[time_id],
                     data_xarray['lat'].values[time_id],10,
                    time_since_release.transpose()[time_id], vmin=0,vmax=10)

clb2 =plt.colorbar(scat2)
clb2.ax.set_xlabel('Age [d]')

t = np.datetime_as_string(timerange[0], unit='m')
title = ax1.set_title('Particles at t = '+t)

def animate(i):
    t = np.datetime_as_string(timerange[i], unit='m')
    title.set_text('Particles at t = '+t)
    
    time_id = np.where(data_xarray['time'] == timerange[i])
    scat1.set_offsets(np.c_[data_xarray['lon'].values[time_id], data_xarray['lat'].values[time_id]])
    scat1.set_array(time_since_release.transpose()[time_id])
    scat2.set_offsets(np.c_[data_xarray['lon'].values[time_id], data_xarray['lat'].values[time_id]])
    scat2.set_array(time_since_release.transpose()[time_id])
 #   scatter = ax.scatter(data_xarray['lon'].values[time_id],
 #                    data_xarray['lat'].values[time_id],10,
 #                   time_since_release.transpose()[time_id], vmin=0,vmax=10)
    
anim = FuncAnimation(fig, animate, frames = len(timerange), interval=500)



In [ ]:
from IPython.display import HTML
HTML(anim.to_jshtml())

In [ ]:
anim.save('003.mp4', fps=15, extra_args=['-vcodec', 'libx264'])

In [ ]:
# subset of particles within a box. 
box_extent = [28.6611, 28.6617, 44.0535,44.0553 ]

def partinbox(i):
    box_id = np.where( (data_xarray['lon'] > box_extent[0])&
                       (data_xarray['lon'] < box_extent[1])&
                       (data_xarray['lat'] > box_extent[2])&
                       (data_xarray['lat'] > box_extent[3])&
                       (data_xarray['time'] == timerange[i]))
    #age = [np.mean(time_since_release.transpose()[box_id]  )]
    ages = time_since_release.transpose()[box_id]
    return(ages)


In [ ]:
labels=[]
ages=[]

for i,t in enumerate(timerange):
    ages.append(partinbox(i))
    labels.append(np.datetime_as_string(t, unit='D')) 


In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_axes([0.03, 0.03, 0.90, 0.94], title='Age Distribution in box')

ax.boxplot(ages,labels=labels)
plt.xticks(rotation='vertical')
ax.set_ylabel('Age [d]')


In [ ]:
from matplotlib.animation import FuncAnimation

# %%capture
fig2 = plt.figure(figsize=(10,10))

ax1 = start_axes('Large', fig=fig2,sp='221', extent=[28, 29.5, 43, 44.5])
ax2 = start_axes('Zoom', fig=fig2,sp='222', extent=[28.6, 28.9, 43.9, 44.5])
ax3 = fig2.add_subplot('212')

time_id = np.where(data_xarray['time'] == timerange[0]) # Indices of the data where time = 0
scat1 = ax1.scatter(data_xarray['lon'].values[time_id],
                     data_xarray['lat'].values[time_id],10,
                    time_since_release.transpose()[time_id], vmin=0,vmax=10)

#clb1 =plt.colorbar(scat1)
#clb1.ax.set_xlabel('Age [d]')

scat2 = ax2.scatter(data_xarray['lon'].values[time_id],
                     data_xarray['lat'].values[time_id],10,
                    time_since_release.transpose()[time_id], vmin=0,vmax=10)

fig2.subplots_adjust(right=0.8)
cbar_ax = fig2.add_axes([0.85, 0.15, 0.05, 0.7])
clb2 = fig2.colorbar(scat2, cax=cbar_ax)
clb2.ax.set_xlabel('Age [d]')

box3  = ax3.boxplot(partinbox(0), labels= [np.datetime_as_string(timerange[0], unit='h')])
ax3.tick_params(labelrotation=90)
ax3.set_ylabel('Age [d]')
ax3.set_xlim( np.datetime_as_string(timerange[0], unit='h') , np.datetime_as_string(timerange[-1], unit='h')     )

t = np.datetime_as_string(timerange[0], unit='m')
title = ax.set_title('Particles at t = '+t)

labels=[]
ages=[]

def animate(i):
    t = np.datetime_as_string(timerange[i], unit='m')
    title.set_text('Particles at t = '+t)
    
    time_id = np.where(data_xarray['time'] == timerange[i])
    scat1.set_offsets(np.c_[data_xarray['lon'].values[time_id], data_xarray['lat'].values[time_id]])
    scat1.set_array(time_since_release.transpose()[time_id])
    scat2.set_offsets(np.c_[data_xarray['lon'].values[time_id], data_xarray['lat'].values[time_id]])
    scat2.set_array(time_since_release.transpose()[time_id])
    
    ages.append(partinbox(i))
    labels.append(np.datetime_as_string(timerange[i], unit='D')) 
    ax3.cla()
    box3  = ax3.boxplot(ages, labels= labels)
    ax3.tick_params(labelrotation=90)
    ax3.set_ylabel('Age [d]')
    ax3.set_title('Age Distribution in box')
    
anim = FuncAnimation(fig2, animate, frames = len(timerange), interval=500)


In [ ]:
labels=[]
ages=[]

HTML(anim.to_jshtml())

In [ ]:
labels=[]
ages=[]
anim.save('003_aged.mp4', fps=5, extra_args=['-vcodec', 'libx264'])